<a href="https://colab.research.google.com/github/olunlo/202106-amazon-scrape/blob/main/20210604_Amazon_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#reference
#https://blog.devgenius.io/how-to-web-scrape-with-python-3a4e38e7a427?gi=d45aa80e1ab5
#https://colab.research.google.com/github/nestauk/im-tutorials/blob/3-ysi-tutorial/notebooks/Web-Scraping/Web%20Scraping%20Tutorial.ipynb
#https://www.youtube.com/watch?v=fPDGu1kQCzE

In [43]:
!pip install beautifulsoup4

In [49]:
!pip install requests

In [56]:
from bs4 import BeautifulSoup
import requests

In [57]:
# Function 1 to get and return parsed HTML using proxy
def get_soup(url):
    # Docker & Splash option (proxy). To run without it, replace with r = requests.get(url)
    res = requests.get('http://localhost:8050/render.html', params = {'url': url, 'wait' : 2})
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

In [58]:
# Initialize list to store reviews data later on
review_list = []

# Define the URLs & URL list (only one for now)
switch_game = 'https://www.amazon.ca/Nintendo-HACPAAACA-Super-Mario-Odyssey/product-reviews/B072V8MMX7/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
url_list = [switch_game]

In [59]:
# Function 2 that sifts through the soup element above, tries to find the tags for the data we want, then appends to the list we initialized
def get_reviews(soup):
    reviews = soup.find_all('div', {'data-hook': 'review'})
    try:
        for item in reviews:
            review = {
            'product': soup.title.text.replace('Amazon.ca:Customer reviews: ', '').strip(), 
            'date': item.find('span', {'data-hook': 'review-date'}).text.strip(),
            'title': item.find('a', {'data-hook': 'review-title'}).text.strip(),
            'rating': float(item.find('i', {'data-hook': 'review-star-rating'}).text.replace('out of 5 stars', '').strip()),
            'body': item.find('span', {'data-hook': 'review-body'}).text.strip(),
            #'helpful': item.find('span', {'data-hook': 'helpful-vote-statement'}).text.replace('found this helpful', '').replace('One', '1').replace('person', '').replace('people', '').strip(), #extra
            }
            review_list.append(review)
    except:
        pass

In [61]:
# Function 3: for every url in url list, run the get_reviews scraper for x many pages
for url in url_list: # url list
    for page in range(1,50): # pages
        soup = get_soup(f'{url}&pageNumber={page}')
        # Amazon product name
        product = soup.title.text.replace('Amazon.ca:Customer reviews: ', '').strip()
        # Print page and product name
        print(f'Getting page {page} reviews for the {product}')
        # Run scraper
        get_reviews(soup)
        # Length of review_list = cumulative results
        c_results = len(review_list)
        # Print cumulative results
        print(f'Cumulative results: {c_results}')
        # Loop for as long as the next page button is not found
        if not soup.find('li', {'class': 'a-disabled a-last'}):
            pass
        else:
            break

ConnectionError: ignored